In [6]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os
import time
import pandas as pd

mingw_bin = "C:/msys64/mingw64/bin"
os.add_dll_directory(mingw_bin)
import ctypes
lin = ctypes.cdll.LoadLibrary("./cmake-build-release/liblinear_model.dll")

lin.create_linear_model.argtypes = [ctypes.c_int32]
lin.create_linear_model.restype = ctypes.c_void_p

lin.predict_linear_model.argtypes = [ctypes.c_void_p, ctypes.POINTER(ctypes.c_float)]
lin.predict_linear_model.restype = ctypes.c_float

lin.release_linear_model.argtypes = [ctypes.c_void_p]
lin.release_linear_model.restype = None

lin.train_linear_model.argtypes = [ctypes.c_void_p, ctypes.POINTER(ctypes.c_float), ctypes.POINTER(ctypes.c_float), ctypes.c_int32, ctypes.c_float, ctypes.c_int32]
lin.train_linear_model.restype = None

lin.create_ovo_classifier.argtypes = [ctypes.c_int32]
lin.create_ovo_classifier.restype = ctypes.c_void_p

lin.train_ovo_classifier.argtypes = [ctypes.c_void_p, ctypes.POINTER(ctypes.c_float), ctypes.POINTER(ctypes.c_int32), ctypes.c_int32, ctypes.c_int32, ctypes.c_float]
lin.train_ovo_classifier.restype = None

lin.predict_ovo_classifier.argtypes = [ctypes.c_void_p, ctypes.POINTER(ctypes.c_float)]
lin.predict_ovo_classifier.restype = ctypes.c_int32

lin.release_ovo_classifier.argtypes = [ctypes.c_void_p]
lin.release_ovo_classifier.restype = None

mlp = ctypes.cdll.LoadLibrary("./cmake-build-release/libmlp.dll") 

mlp.create_mlp_model.argtypes = [ctypes.POINTER(ctypes.c_int), ctypes.c_int, ctypes.c_int]
mlp.create_mlp_model.restype = ctypes.c_void_p

mlp.train_mlp_model.argtypes = [
    ctypes.c_void_p,
    ctypes.POINTER(ctypes.c_float),
    ctypes.POINTER(ctypes.c_float),
    ctypes.c_int,
    ctypes.c_int,
    ctypes.c_float,
    ctypes.c_int,
    ctypes.POINTER(ctypes.c_float),
    ctypes.POINTER(ctypes.c_float),
    ctypes.c_int,
    ctypes.c_char_p
]
mlp.predict_mlp_model.restype = ctypes.POINTER(ctypes.c_float)

mlp.predict_mlp_model.argtypes = [
    ctypes.c_void_p,
    ctypes.POINTER(ctypes.c_float),
]

mlp.release_mlp_model.argtypes = [ctypes.c_void_p]

mlp.get_confusion_matrix.argtypes = [ctypes.c_void_p, ctypes.POINTER(ctypes.c_int32)]
mlp.get_confusion_matrix.restype = None

mlp.evaluate_confusion_matrix.argtypes = [
    ctypes.c_void_p,
    ctypes.POINTER(ctypes.c_float),
    ctypes.POINTER(ctypes.c_float),
    ctypes.c_int
]
mlp.evaluate_confusion_matrix.restype = None

mlp.save_mlp_model.argtypes = [ctypes.c_void_p, ctypes.c_char_p]

mlp.load_mlp_model.argtypes = [ctypes.c_char_p]
mlp.load_mlp_model.restype = ctypes.c_void_p


LOG_FUNC_TYPE = ctypes.CFUNCTYPE(None, ctypes.c_char_p)

@LOG_FUNC_TYPE
def logger_callback(msg):
    print(msg.decode())

In [7]:
classnames_array = ['Crocodile', 'Elephant', 'Gorille']
size = 32

### Organisation du Dataset pour l'entrainement

In [8]:
data = []
folder_path = './images32/'
for classname in os.listdir(folder_path):  
    for filename in os.listdir(folder_path + classname):
        img = Image.open(folder_path + classname + '/' + filename)
        index = classnames_array.index(classname)  
        
        one_hot = np.zeros(len(classnames_array), dtype=np.float32)
        one_hot[index] = 1.0
        
        img_array = (np.array(img) / 255.0) - 1.0
        data.append([one_hot, img_array])

data = np.array(data, dtype=object)




### Test avec MLP

In [ ]:
layer_sizes = [size*size*3, 256, 128, len(classnames_array)]
layer_array = (ctypes.c_int * len(layer_sizes))(*layer_sizes)
model = mlp.create_mlp_model(layer_array, len(layer_sizes) - 1, 20)
mlp.set_logger(logger_callback)

np.random.shuffle(data)
split_index = int(len(data) * 0.8)
data_train = data[:split_index]
data_test = data[split_index:]
X_train = np.array([item[1] for item in data_train], dtype=np.float32)
Y_train = np.array([item[0] for item in data_train], dtype=np.float32)
X_test = np.array([item[1] for item in data_test], dtype=np.float32)
Y_test = np.array([item[0] for item in data_test], dtype=np.float32)

X_train_reshaped = X_train.reshape((X_train.shape[0], -1))
X_train_c = X_train_reshaped.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
Y_train_c = Y_train.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
X_test_reshaped = X_test.reshape((X_test.shape[0], -1))
X_test_c = X_test_reshaped.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
Y_test_c = Y_test.ctypes.data_as(ctypes.POINTER(ctypes.c_float))

epochs = 15000
learning_rate = 0.005
batch_size = 32
csv_path = "log2.csv"
with open(csv_path, "w") as csvfile:
        csvfile.truncate()
        csvfile.write("epoch,train_acc,train_loss,test_acc,test_loss\n")
time.sleep(0.2)

csv_path = csv_path
mlp.train_mlp_model(model, X_train_c, Y_train_c, len(data_train), epochs, learning_rate, batch_size, X_test_c, Y_test_c, len(data_test), csv_path.encode())

output_dim = len(classnames_array)
matrix = (ctypes.c_int * (output_dim * output_dim))()
mlp.evaluate_confusion_matrix(model, X_test_c, Y_test_c, len(data_test), True)
mlp.get_confusion_matrix(model, matrix)

confusion_np = np.ctypeslib.as_array(matrix).reshape((output_dim, output_dim))
print(confusion_np)
import seaborn as sns


plt.figure(figsize=(8, 6))
sns.set(font_scale=1.2)
ax = sns.heatmap(confusion_np, annot=True, fmt='d', cmap='Blues', 
                 xticklabels=classnames_array, yticklabels=classnames_array,
                 linewidths=1, linecolor='gray', cbar=True)

plt.title("Matrice de confusion", fontsize=16)
plt.xlabel("Classe prédite", fontsize=14)
plt.ylabel("Classe réelle", fontsize=14)
plt.tight_layout()
plt.show()

import matplotlib.pyplot as plt

df = pd.read_csv("log.csv")

plt.figure(figsize=(10, 5))
plt.plot(df['epoch'], df['train_loss'], label='Train Loss', color='blue')
plt.plot(df['epoch'], df['test_loss'], label='Test Loss', color='red')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Train vs Test Loss')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 5))
plt.plot(df['epoch'], df['train_acc'], label='Train Accuracy', color='green')
plt.plot(df['epoch'], df['test_acc'], label='Test Accuracy', color='orange')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Train vs Test Accuracy')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

TypeError: bytes or integer address expected instead of str instance

In [ ]:
mlp.release_mlp_model(model)

1

In [ ]:
mlp.save_mlp_model(model, b"mlp_trained.bin")

-201418088

In [ ]:
model_loaded = mlp.load_mlp_model(b"mlp_trained.bin")

OSError: [WinError -1073741569] Windows Error 0xc00000ff

In [ ]:
img = Image.open('C:/Users/lgrdp/Downloads/images (10).jpeg').convert("RGB")
img = img.resize(size=(32,32))
img_data = np.array(img) / 255.0

output_array = mlp.predict_mlp_model(model, img_data.ctypes.data_as(ctypes.POINTER(ctypes.c_float)), True)
output_array = ctypes.cast(output_array, ctypes.POINTER(ctypes.c_float * len(classnames_array))).contents
output = list(output_array)
class_index = 0
for i in range(len(output)):
    if output[i] == max(output):
        class_index = i
print('predicted class : ' + classnames_array[class_index])
print(output)


predicted class : zebre
[0.956343948841095, 0.983494222164154, 0.9977808594703674]
